In [8]:
%load_ext autoreload
%autoreload 2

In [5]:
REPO_NAME = 'sewer-nfl'
import sys, os
cwd = str(os.getcwd())
repo_dir = cwd[:cwd.find(REPO_NAME)+len(REPO_NAME)]
sys.path.insert(0,repo_dir)

In [1]:
import pandas as pd

In [6]:
import sys
sys.path

['c:\\Users\\zacha\\Documents\\GitHub\\sewer-nfl',
 'c:\\Users\\zacha\\Documents\\GitHub\\sewer-nfl\\warehouse\\notebooks',
 'C:\\Users\\zacha\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip',
 'C:\\Users\\zacha\\AppData\\Local\\Programs\\Python\\Python310\\DLLs',
 'C:\\Users\\zacha\\AppData\\Local\\Programs\\Python\\Python310\\lib',
 'C:\\Users\\zacha\\AppData\\Local\\Programs\\Python\\Python310',
 'c:\\Users\\zacha\\Documents\\GitHub\\sewer-nfl\\.venv',
 '',
 'c:\\Users\\zacha\\Documents\\GitHub\\sewer-nfl\\.venv\\lib\\site-packages',
 'c:\\Users\\zacha\\Documents\\GitHub\\sewer-nfl\\.venv\\lib\\site-packages\\win32',
 'c:\\Users\\zacha\\Documents\\GitHub\\sewer-nfl\\.venv\\lib\\site-packages\\win32\\lib',
 'c:\\Users\\zacha\\Documents\\GitHub\\sewer-nfl\\.venv\\lib\\site-packages\\Pythonwin']

In [7]:
import warehouse.pipelines.game_summary.game_summary